In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot = True)

C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hansh\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_input = 28*28
n_hidden = 256
n_noise = 128
X=tf.placeholder(tf.float32,[None,n_input])
Z=tf.placeholder(tf.float32,[None,n_noise])

In [3]:
G_W1 = tf.Variable(tf.random_normal([n_noise,n_hidden],stddev=0.01))
G_b1 = tf.Variable(tf.random_normal([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden,n_input],stddev =0.01))
G_b2 = tf.Variable(tf.random_normal([n_input]))
D_W1 = tf.Variable(tf.random_normal([n_input,n_hidden],stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden,1],stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z,G_W1)+G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,G_W2)+G_b2)
    return output
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs,D_W1)+D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2)
    return output
def get_noise(batch_size,n_noise):
    return np.random.normal(size=(batch_size,n_noise))

In [5]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)
loss_D = tf.reduce_mean(tf.log(D_real)+tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))
D_var_list = [D_W1,D_b1,D_W2,D_b2]
G_var_list = [G_W1,G_b1,G_W2,G_b2]

In [6]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,var_list=G_var_list)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D,loss_val_G = 0,0
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size,n_noise)
        _,loss_val_D = sess.run([train_D,loss_D],feed_dict={X:batch_xs,Z:noise})
        _,loos_val_G = sess.run([train_G,loss_G],feed_dict={Z:noise})
    print('Epoch:','%04d'%epoch,'D loss:','{:.4f}'.format(loss_val_D),'G loss: ','{:.4f}'.format(loss_val_G))
    if epoch == 0 or (epoch+1)%10 == 0:
        sample_size = 10
        noise = get_noise(sample_size,n_noise)
        samples = sess.run(G,feed_dict={Z:noise})
        fig,ax =plt.subplots(1,sample_size,figsize=(sample_size,1))
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i],(28,28)))
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)),bbox_inches='tight')
        plt.close(fig)

Epoch: 0000 D loss: -0.8511 G loss:  0.0000
Epoch: 0001 D loss: -0.6579 G loss:  0.0000
Epoch: 0002 D loss: -0.2177 G loss:  0.0000
Epoch: 0003 D loss: -0.8239 G loss:  0.0000
Epoch: 0004 D loss: -0.4470 G loss:  0.0000
Epoch: 0005 D loss: -0.6793 G loss:  0.0000
Epoch: 0006 D loss: -0.4929 G loss:  0.0000
Epoch: 0007 D loss: -0.5864 G loss:  0.0000
Epoch: 0008 D loss: -0.4373 G loss:  0.0000
Epoch: 0009 D loss: -0.3810 G loss:  0.0000
Epoch: 0010 D loss: -0.3519 G loss:  0.0000
Epoch: 0011 D loss: -0.2787 G loss:  0.0000
Epoch: 0012 D loss: -0.2280 G loss:  0.0000
Epoch: 0013 D loss: -0.2662 G loss:  0.0000
Epoch: 0014 D loss: -0.3637 G loss:  0.0000
Epoch: 0015 D loss: -0.1573 G loss:  0.0000
Epoch: 0016 D loss: -0.1751 G loss:  0.0000
Epoch: 0017 D loss: -0.3459 G loss:  0.0000
Epoch: 0018 D loss: -0.3888 G loss:  0.0000
Epoch: 0019 D loss: -0.2754 G loss:  0.0000
Epoch: 0020 D loss: -0.3843 G loss:  0.0000
Epoch: 0021 D loss: -0.2225 G loss:  0.0000
Epoch: 0022 D loss: -0.4025 G lo